# Variant effect prediction analysis


In [13]:
import polars as pl
from scipy.stats import chi2


In [ ]:
lead_variant_maf_dataset = pl.read_parquet("../../lead_variant_maf.parquet")


In [ ]:
lead_variant_maf_dataset.shape


(2622098, 17)

In [ ]:
lead_variant_maf_dataset.head()


studyId,studyLocusId,variantId,beta,zScore,pValueMantissa,pValueExponent,standardError,finemappingMethod,studyType,credibleSetSize,nCases,nControls,nSamples,majorPopulation,majorPopulationMAF,majorPopulationAlleleFrequency
str,str,str,f64,f64,f32,i32,f64,str,str,i64,i32,i32,i32,struct[2],f64,list[struct[2]]
"""gtex_leafcutter_nerve_tibial_1…","""c67d0c268923d0eb68ee2cab99f602…","""13_23633308_G_C""",-0.395991,null,3.371,-9,0.0658143,"""SuSie""","""sqtl""",43,null,null,532,"{""nfe"",0.0}",0.277434,"[{""nfe_adj"",0.277434}]"
"""twinsuk_exon_skin_ensg00000289…","""2d630434f5d1a103382ac557c2d5d5…","""13_23633813_C_T""",1.71288,null,1.332,-12,0.232885,"""SuSie""","""eqtl""",4,null,null,370,"{""nfe"",0.0}",0.022223,"[{""nfe_adj"",0.022223}]"
"""gtex_leafcutter_skin_sun_expos…","""3353619fc6dd5cafa0226e08a0c3b3…","""13_23635163_A_T""",-0.559313,null,6.588,-19,0.0608432,"""SuSie""","""sqtl""",14,null,null,602,"{""nfe"",0.0}",0.282794,"[{""nfe_adj"",0.282794}]"
"""peng_2018_exon_placenta_naive_…","""5834163a41e2bc3b55605b945048e4…","""13_23638347_G_A""",0.619266,null,5.508,-13,0.0775104,"""SuSie""","""eqtl""",9,null,null,149,"{""nfe"",0.0}",0.320347,"[{""nfe_adj"",0.320347}]"
"""peng_2018_ge_placenta_naive_en…","""dab737df0056b013c999db7b392ab3…","""13_23640464_G_A""",0.630709,null,4.972,-14,0.0748735,"""SuSie""","""eqtl""",8,null,null,149,"{""nfe"",0.0}",0.319922,"[{""nfe_adj"",0.319922}]"


In [20]:
df = lead_variant_maf_dataset.select(
    (pl.col("pValueMantissa") * 10).pow(pl.col("pValueExponent")).alias("pValue"),
    (pl.col("pValueMantissa") * 10)
    .pow(pl.col("pValueExponent").map_elements(lambda x: chi2.cdf(x, 1), pl.Float64()))
    .alias("cdf"),
    pl.col("variantId"),
    pl.col("nSamples"),
    pl.col("studyType"),
).select(
    "variantId",
    "studyType",
    "cdf",
    "pValue",
    (pl.col("cdf") / pl.col("nSamples")).alias("varianceExplained"),
)


In [ ]:
df


pValue,cdf
f32,f32
1.7790e-14,1.0
3.2059e-14,1.0
2.7777e-35,1.0
2.3285e-23,1.0
1.7724e-24,1.0
